<h1><font color="red">Still need to be improved!</font></h1>
<p>The extraction of party information remains inaccurate and manully labelling is still a huge work to be done.</p>

In [1]:
#! /usr/bin/env python3
import pandas as pd
import numpy as np
import os
import json
import spacy
# import zh_core_web_sm
# import en_core_web_sm
# import zh_core_web_trf
import zh_core_web_lg
from spacy.matcher import Matcher
from spacy.matcher import PhraseMatcher
from spacy.tokens import Span
from spacy.language import Language
from spacy.attrs import ORTH
import re
from spacy.tokens import Doc
from spacy.pipeline import EntityRuler
from spacy import displacy
from spacy.tokenizer import Tokenizer
from spacy.symbols import POS
from spacy.strings import StringStore
from spacy.pipeline import Tagger

In [2]:
total_basic_url = "/Users/starice/Desktop/"

#处理网上爬下来的json
with open('/Users/starice/OwnFiles/cityu/RA/type1/2014/12/json/57ac657ac2265c25898b2240.json', 'r', encoding="UTF-8") as f:
    data = json.load(f)
# print(data)
# print(type(data))
dict_data = json.loads(data)

In [3]:
# 使用指定的string list作为matcher词库去匹配
party_titles = ['三被告共同委托代理人', '两被申诉人的共同委托代理人', '被上诉人(原审原告、反诉被告)', \
                '上述两被告的共同委托诉讼代理人', '代表人', '公司代表人', '法定代表人', '特别授权被告', \
                '上诉人(原审第三人)', '上述两被告委托代理人', '上述两被告共同委 托诉讼代理人', \
                '再审申请人(原审被告)', '委托代理人(特别授权)', '被申请人(一审被告、二审被上诉人)', \
                '委托诉讼代理人(特别授权)', '被告委托代理人', '再审申请人', '诉讼委托代理人', '被告一', \
                '拟稿人', '被上诉人(原审被告)', '上诉人(原审原告)', '被上诉人二(原审被告二)', \
                '两被告共同委托代理人', '上诉人共同委托代理人', '原审被告', '被告二', '上诉人(原审被告、反诉原告)', \
                '三上诉人的共同委托代理人', ' 再审申请人(一审原告)', '申诉人(一审原告、二审上诉人)', \
                '支持起诉人', '两上诉人共同委托的诉讼代理人', '原审第二被告', '被上诉人一(原审被告一)', \
                '一审被告', '原告', '被上诉人共同委托代理人', '原审第三人 ', '两原审被告共同委托代理人', \
                '被告', '上述三上诉人共同委托代理人', '被上诉人(一审原告)', '授权代理人', '诉讼代表人', \
                '被告(反诉原告)', '法人代表', '法人代表人', '法定代表人(负责人)', \
                '两被上诉人共同的委托代理人', '上述两被告的委托 诉讼代理人', '申诉人(一审原告、二审被上诉人)', \
                '(一审被告、二审被上诉人)', '一审第三人', '上诉人', '以上原告共同委托代理人', \
                '以上二被上诉人的共同委托诉讼代理人', '申请再审人(原审被告)', '公益诉讼出庭人', \
                '以上二被告共同委托代理人', '以上三被告共同委托代理人', '上述两被告的共同委托代理人', \
                '以上两被告委托代理人', '被申诉人(一审被告、二审上诉人)', '两被上诉人的共同委托代理人', \
                '二被告之共同委托诉讼代理人', '共同委托诉讼代理人', '两被告的共同委托代理人', '支持起诉机关', \
                '上列两被上诉人委托代理人 ', '上述两被上诉人的共同委托诉讼代理人', '二被告共同委托诉讼代理人', \
                '上列两被上诉人的委托代理人', ' 两被告共同的委托代理人', '上述两被告共同委托代理人', \
                '被上诉人的共同委托诉讼代理人', '以上三被告共同的委托诉讼代理人', '抗诉机关', '被上诉人', \
                '原审第三被告', '上诉人(一审原告)', '二被告共同委托代理人', '以上两被告共同委委托代理人', \
                '两被申诉人共同委托诉讼代理人', '特别授权委托诉讼代理人', '两上诉 人共同委托代理人', \
                '以上两被上诉人共同委托诉讼代理人', '上诉人(原审第一被告)', '申诉人(一审原告、二审上诉人、原再审申请人)', \
                '第二被告', '被申请人(原审被告)', '上述两上诉人共同委托代理人', \
                '被申诉人 (一审被告、二审被上诉人、原再审被申请人)', '再审申请人(一审原告、二审上诉人)', \
                '两上诉人的共同委托代理人', '上列两被告的共同委托代理人', '上列两被告共同委托代理人', \
                '被申请人(一审被告)', '原告共同委托诉讼代理人', '被申请人(一审被告、二审上诉人)', \
                '上诉人共同委托诉讼代理人', '被申诉人(一审被告、二审 被上诉人)', '被上诉人(一审被告)', \
                '上述二上诉人的共同委托诉讼代理人', '被申诉人(原审被告)', '二被上 诉人共同委托诉讼代理人', \
                '上述两上诉人共同的委托诉讼代理人', '第三被告', '再审申请人(一审原告、二审被 上诉人)', \
                '两上诉人共同委托诉讼代理人', '被申请人(一审原告、二审被上诉人)', '申诉人(原审原告)', \
                '两被上诉人共同委托诉讼代理人', '诉讼代理人', '上诉人(一审被告)', '被申请人(原审原告)', \
                '上诉人(原审被告)', '以上二上诉人共同的委托诉讼代理人', '上述两上诉人的共同委托代理人', \
                '两上诉人的共同委托诉讼代理人', '上述两被上诉人委托诉讼代理人', '上上诉人(原审被告)', \
                '第三人', '法定代理人', '被上诉人(原审第三人)', '原告(反诉被告)', '第一被告', \
                '三被上诉人共同委托诉讼代理人', '委托代理人', '被申诉人(一审被告、二审被上诉人、再审被申请人)', \
                '委托代理人(特别授权代理)', '两第三人的共同委托代理人', '被上诉人共同委托代理人(特别授权代理)', \
                '再审申请人(一审被告、二审上诉人)', '以上两被告共同委托 代理人', '以上两被告的委托代理人', \
                '上述两上诉人共同委托诉讼代理人', '上述被上诉人共同委托诉讼代理人 ', \
                '二被委托代理人', '被申请人', '两被告共同委托诉讼代理人', '公益诉讼起诉人', \
                '被上诉人(原审原告)', '上述两被上诉人共同委托代理人', '委托诉讼代理人', '上述被告共同委托诉讼代理人', \
                '以上二被上诉 人委托代理人', '上列两被上诉人的共同委托代理人', '共同委托代理人', \
                '再审申请人(原审原告)', '上述上诉人的委托诉讼代理人', '申诉人(一审原告、二审上诉人、再审申请人)', \
                '二被上诉人的委托代理人', '上述两被上诉人的共同委托代理人', '上列二被告共同委托代理人', \
                '以上两被告共同委托诉讼代理人', '执行事务合伙人', '投资人', '以上两被告的共同委托代理人', 
                '两被上诉人之共同委托代理人', '再审申请人(一审被告、二审被上诉人)', '被申请人(一审原告、二审上诉人)', \
               '一审被告、二审被上诉人', '再审申请人:(一审第三人、二审上诉人)', '申请再审人(一审原告、二审上诉人)', \
               '被申请人(一审被告,二审被上诉人)', '被申诉人(一审被告、二审被上诉人)', 
                '被申诉人(一审被告、二审被上诉人、原再审被申请人)', \
                '上诉人(原告)', '上诉人(被告)', '原审当事人(原审被告)'
               ]
plaintiff_titles = ['上诉人', '上诉人(一审原告)', '上诉人(一审第三人)', '上诉人(原告)', '上诉人(原审原告)', '上诉人(原审原告、反诉被告)', 
                   '上诉人(原审原告人)', '上诉人(原审第三人)', '公益诉讼起诉人', '再审申请人', '再审申请人(一审原告)',
 '再审申请人(一审原告、二审上诉人)',
 '再审申请人(一审原告、二审被上诉人)', '再审申请人(原审原告)',
 '再审申请人(原审原告、二审上诉人)', '再审申请人:(一审第三人、二审上诉人)', '原告',
 '原告(反诉被告)', '抗诉机关', '支持起诉人',
 '支持起诉机关', '申诉人(一审原告、二审上诉人)',
 '申诉人(一审原告、二审上诉人、再审申请人)',
 '申诉人(一审原告、二审上诉人、原再审申请人)',
 '申诉人(一审原告、二审被上诉人)',
 '申诉人(原审原告)',
 '申请再审人(一审原告、二审上诉人)', '被上诉人(一审原告)', '被上诉人(原审原告)',
 '被上诉人(原审原告、反诉被告)', '被上诉人(原甲原告)',
 '被上诉人一(原审原告)',
 '被上诉人一(原审被告一)',
 '被上诉人二(原审被告二)', '被申请人(一审原告、二审上诉人)',
 '被申请人(一审原告、二审被上诉人)', '被申请人(原审原告)']

defendant_titles = ['(一审被告、二审被上诉人)', '一审被告', '一审被告(二审上诉人)', '一审被告、二审被上诉人', '一审被告二审上诉人)', 
                   '上上诉人(原审被告)', '上诉人(一审被告)', '上诉人(原审第一被告)', '上诉人(原审被告)',
 '上诉人(原审被告、反诉原告)',
 '上诉人(原审被告一)', '上诉人(被告)', '公益诉讼出庭人', '再审申请人(一审被告)',
 '再审申请人(一审被告、二审上诉人)',
 '再审申请人(一审被告、二审被上诉人)', '再审申请人(再审被告)', '再审申请人(原审被告)', '原审当事人(原审被告)',
 '原审第三被告',
 '原审第二被告',
 '原审被告',
 '原审被告(反诉原告)', '特别授权被告', '申请再审人(原审被告)',
 '第一被告', '第三被告',
 '第二被告',
 '被上诉人',
 '被上诉人(一审被告)', '被上诉人(原审第三人)',
 '被上诉人(原审被告)',
 '被上诉人(原审被告、反诉原告)',
 '被上诉人(原审被告人)', '被告',
 '被上诉人(原甲被告)',
 '被告(反诉原告)',
 '被告一',
 '被告二', '被申诉人(一审被告,二审被上诉人)',
 '被申诉人(一审被告、二审上诉人)',
 '被申诉人(一审被告、二审被上诉人)',
 '被申诉人(一审被告、二审被上诉人、再审被申请人)',
 '被申诉人(一审被告、二审被上诉人、原再审被申请人)',
 '被申诉人(原审被告)',
 '被申请人', '被申请人(一审被告)',
 '被申请人(一审被告,二审被上诉人)',
 '被申请人(一审被告、二审上诉人)',
 '被申请人(一审被告、二审被上诉人)', '被申请人(原审被告)', '被申请人(原审被告、二审被上诉人)']

party_titles = list(set(party_titles + plaintiff_titles + defendant_titles))

party_infos = ['身份证地址', '联系地址', '通信地址', '现住', '住', '营业场所', '信用代码', '籍贯', \
               '住址', '住址地', '所在地', '户籍所在地', '住所', '营业地', '身份证住址', \
               '港澳证件号码', '原名称', '注册号', '组织机构代码', '非公司私营企业住所', \
               '户籍地址', '身份证号码', '组织机构代码证', '公⺠身份号码', '营业执照', \
               '身份号码', '营业执照注册号', '住所地', '执照注册号', '经营场所', '身份证住址', \
               ' 个体工商户营业执照注册', '机构代码', '户籍住所', '统一信用代码', '原名称', \
               '身份证地址', '经营地址', '身份证登记住址', '组织机构代码证号', '执业证号', \
               '曾用名', '统一社会信用代码', '工商注册号', '身份证号', '注册号', '营业场所', \
               '居⺠身份证', '地址', '现住址', '经常居住地', '代理权限', '户籍地', \
               '营业执照号码', '公⺠身份证号', '中文名称 ', '所在地', '户籍地址', \
               '组织机构代码证', '公⺠身份号码', '代码', '经营业主', '经营场所', \
               '身份证登记住址', '统一社会信用代码', '地址', '注册号', '营业场所', '居⺠身份证号码', \
               '户籍地', '身份证住址', '公民身份号码', '公民身份证号码']
    
party_occupations = ['律师', '经理', '总经理', '董事长', '法务', '无职业', '业主', '员工', '店长', \
                     '主管', '经营者', '董事', '职员', '部长', '负责人', '法律工作者']



# TODO 赔偿款也有10倍货款或十倍货款这种形式
# 返还一般都是货款，赔偿一般都是赔偿金，再优化一下方法！
cost_type = ['货款','价款','受理费','赔偿金','公证费','诉讼费','购物款','上诉费','公告费', '运费', 
             '交通费','误工费','打印费','鉴定费','邮寄费','赔偿款','医疗费','购酒款','购药款', 
             '住宿费','产品质量监督检验费','其他费用', '餐饮费', '购货款', '减半', '合计', '共计', 
             '管辖权异议费', '十倍价款', '十倍货款', '价款的十倍', '货款的十倍', '十倍购物款', '餐费', 
             '检测费', '三倍货款']

judge_titles = ['审判长', '审判员', '人民审判员', '助理审判员', '代理审判员', '人民陪审员']

In [4]:
#处理当事人信息时可以被从括号中去掉的内容，在匹配了所有的title之后去掉其他文本的括号内容
# 例如：
# (代理以上两被告)
# (代理上述二被告)
# (原告孙桂兰之子)
# (原告李娟之夫)


In [5]:
# 定义一个合并字典值的工具类
def merge_dict(x,y):
    for k,v in x.items():
                if k in y.keys():
                    y[k] += v
                else:
                    y[k] = v
    return y
case_truth_relist = {'case_truth_re_1':1, 'case_truth_re_2':2, 'case_truth_re_3':3, 'case_truth_re_4':0, 'case_truth_re_5':0}
court_opin_relist = {'case_truth_re_1':0, 'case_truth_re_2':2, 'case_truth_re_3':1}
print(merge_dict(court_opin_relist, case_truth_relist))

{'case_truth_re_1': 1, 'case_truth_re_2': 4, 'case_truth_re_3': 4, 'case_truth_re_4': 0, 'case_truth_re_5': 0}


In [6]:
#还有各种之姐， 之夫， 之父， 之母， 之妻， 之子， 之女， 之， 的， 系， 是， 原告提供， 需要补充
invaid_partytitle = ["系原告", "与原告", "是原告", 
                       "和原告", "为原告", "系被告", 
                       "与被告", "是被告", "和被告", 
                       "为被告", "原告之", "被告之", 
                       "原告的", "被告的"]
invalid_partyname = [
    "之姐", "之夫", "之父", "之母", "之妻", 
    "之子", "之女", "儿媳", "婆婆", "丈夫", 
    "儿子", "女儿", "公公", "外甥", "表哥"
]
partytitle_token_pattern = [{"TEXT": {"IN": list(party_titles), "NOT_IN": list(invaid_partytitle)}}]
partyinfo_token_pattern = [{"TEXT": {"IN": list(party_infos)}}]
partyoccup_token_pattern = [{"TEXT": {"IN": list(party_occupations)}}]

In [7]:
# reasons可以大致分为几类： refer ReasonKeywords for details
ct_pattern_1_1 = [{"ORTH": {"IN": ["不安全", "不合格", "假冒"]}}, {"OP": "*"}, {"ORTH": {"IN": ["食品", "产品"]}}]
ct_pattern_1_2 = [{"ORTH": {"IN": ["产品", "食品"]}}, {"OP": "*"}, {"ORTH": {"IN": ["不合格", "不符合"]}}]

ct_pattern_2 = [{"ORTH": {"IN": ['国家安全标准食品添加剂使用标准', '饮料通则', '预包装饮料酒标签通则']}}]

ct_pattern_3 = [{"ORTH": {"IN": ["不符合", "违反", "无"]}}, {"OP": "*"}, {"ORTH": {"IN": ["食品安全", "安全食用标准", "安全标准", "质量安全"]}}]
# ct_pattern_3 = [{"ORTH": {"REGEX": "违反"}}, {"OP": "*"}, {"ORTH": {"REGEX": "食品安全"}}]

ct_pattern_4_1 = [{"ORTH": {"IN": ['产品标准代号', '产品说明', '标签', '标识', '预包装饮料酒标签通则']}}]
ct_pattern_4_2 = [{"ORTH": {"IN": ["未标注", "没有标注"]}}, {"OP": "*"}, {"ORTH": "成分"}]

ct_pattern_5 = [{"ORTH": {"IN": ['保质期', '生产日期', '过期']}}]

ct_pattern_6 = [{"ORTH": {"IN": ['原始配料', '原料', '原材料', '非药食同源物质', '添加剂', '非食用物质', '食品原料']}}]

ct_pattern_7 = [{"ORTH": "欺诈"}]

ct_pattern_8 = [{"ORTH": {"IN": ['商标专用权', '商标违法']}}]

ct_pattern_9 = [{"ORTH": "出入境检验"}]

ct_pattern_10 = [{"ORTH": {"IN": ['生产许可证编号', '生产许可编号', '生产许可证']}}]

In [8]:
# nlp = zh_core_web_trf.load()
nlp = spacy.load("zh_core_web_lg")
patterns_ptitles = list(nlp.pipe(party_titles))
patterns_pinfos = list(nlp.pipe(party_infos))
patterns_poccups = list(nlp.pipe(party_occupations))
# patterns_jtitles = list(nlp.pipe(judge_titles))
patterns_jtitles = [nlp.make_doc(text) for text in judge_titles]

In [9]:
# case reasons matcher
ct_matcher = Matcher(nlp.vocab)
ct_matcher.add("不合格产品", [ct_pattern_1_1])
ct_matcher.add("不合格产品", [ct_pattern_1_2])
ct_matcher.add("违反条款", [ct_pattern_2])
ct_matcher.add("违反食品安全标准", [ct_pattern_3])
ct_matcher.add("标签", [ct_pattern_4_1])
ct_matcher.add("标签", [ct_pattern_4_2])
ct_matcher.add("保质期、生产日期", [ct_pattern_5])
ct_matcher.add("原材料、添加剂", [ct_pattern_6])
ct_matcher.add("欺诈", [ct_pattern_7])
ct_matcher.add("商标违法", [ct_pattern_8])
ct_matcher.add("出入境检验", [ct_pattern_9])
ct_matcher.add("生产许可证", [ct_pattern_10])

In [10]:
# Append words to user dict(like jieba.add_word) 帮助分词， 抛弃原先用jieba分词结果代替spacy分词的方法
# nlp.tokenizer.pkuseg_update_user_dict(party) #这一步移到了处理单个case file方法内，避免大量的文件读取
nlp.tokenizer.pkuseg_update_user_dict(cost_type)
nlp.tokenizer.pkuseg_update_user_dict(party_titles)
nlp.tokenizer.pkuseg_update_user_dict(party_infos)
nlp.tokenizer.pkuseg_update_user_dict(judge_titles)
#细微分词调节
judgeresult_verbs = ["赔偿", "退还", "负担", "返还", "支付", 
                     "赔付", "承担", "收取", "迳付", "给付", 
                     "交纳", "缴纳", "担负", "退款"]
nlp.tokenizer.pkuseg_update_user_dict(judgeresult_verbs)
nlp.tokenizer.pkuseg_update_user_dict(["本判决", "发生法律效力之日起", "发生法律效力后", 
                                       "本判决生效之日起", "本判决生效后", "于", "内",
                                       "向", "计", "本判决书生效后"])
nlp.tokenizer.pkuseg_update_user_dict(["系原告", "与原告", "是原告", "和原告", "为原告", 
                                       "系被告", "与被告", "是被告", "和被告", "为被告", 
                                      "原告之", "被告之"])
nlp.tokenizer.pkuseg_update_user_dict(["不安全", "不合格", "不符合", 
                                       "国家安全标准食品添加剂使用标准", "饮料通则", 
                                       "预包装饮料酒标签通则", "没有标注",
                                       "未标注", "产品标准代号", "产品说明", "原始配料", 
                                       "非药食同源物质", "非食用物质", "食品原料", 
                                       "商标专用权", "商标违法", "出入境检验", "生产许可证编号", 
                                       "生产许可编号", "生产许可证", "食品安全", 
                                       "安全食用标准", "安全标准", "质量安全"])

In [11]:
# 使用entityruler帮助nlp识别特定的实体(费用类型、当事人等等)
# Entity Ruler V3 (The use of add_pipe was changed because of the version update of spacy)
# Add necessary patterns to global entityruler(including party_titles, party_infos, cost_types, judge_verbs from lists)
# global_entityruler = EntityRuler(nlp, overwrite_ents=True)
if "global_entityruler" in list(nlp.pipe_names):
    nlp.remove_pipe("global_entityruler")
global_entityruler = nlp.add_pipe("entity_ruler", name="global_entityruler", after="ner")
patterns_cost_type = [{"label": "COST_TYPE", "pattern": str(i)} for i in cost_type]
patterns_party_title = [{"label": "PARTY_TITLE", "pattern": str(i)} for i in party_titles]
patterns_party_info = [{"label": "PARTY_INFO", "pattern": str(i)} for i in party_infos]
patterns_judgere_verbs = [{"label": "JUDGE_VERB", "pattern": str(i)} for i in judgeresult_verbs]
global_entityruler.add_patterns(patterns_cost_type)
global_entityruler.add_patterns(patterns_party_title)
global_entityruler.add_patterns(patterns_party_info)
global_entityruler.add_patterns(patterns_judgere_verbs)
# nlp.add_pipe("global_entityruler", after="ner")

In [12]:
#使用自定义的pos mapping table来对一些词汇指定词性
attribute_ruler = nlp.get_pipe("attribute_ruler")
tagger_pattern = [{"TEXT": {"IN": list(judgeresult_verbs)}}]
tagger_attrs1 = {"POS": "VERB"}
attribute_ruler.add(patterns=[tagger_pattern], attrs=tagger_attrs1)

In [13]:
doc_test = nlp("原告要求被告退还货款991.02元,本院予以支持**")
print(doc_test[3].text, doc_test[3].tag_, doc_test[3].pos_) 

退还 VV VERB


In [14]:
#使用regex和spacy self-defined entity recognizer class来帮助识别金额实体
money_re = r"(?P<money_amount>([零一二三四五六七八九十百千万亿角元\d+\.,]*)(?=[角|元])[角|元])"

# animal_hash = StringStore([u'MONEY_AMOUNT']) # <-- match id
nlp.vocab.strings.add('MONEY_AMOUNT')

@Language.factory("money_amount", default_config={"label": "MONEY_AMOUNT", "name": "money_amount"})
class MoneyRecognizer:
    def __init__(self, nlp, name="money_amount", label="MONEY_AMOUNT"):
        self.nlp = nlp
        self.name = name
        self.label = nlp.vocab.strings[label]  # get entity label ID

    def __call__(self, doc):
        """Apply the pipeline component on a Doc object and modify it if matches
        are found. Return the Doc, so it can be processed by the next component
        in the pipeline, if available.
        """
        
        expression = money_re
#         spans = []  # keep the spans for later so we can merge them afterwards
        for match in re.finditer(expression, doc.text):
            start, end = match.span()
            span = doc.char_span(start, end, label=self.label)
            # This is a Span object or None if match doesn't map to valid token sequence
            if span is not None:
#                 spans.append(span)
                doc.ents = list(doc.ents) + [span]
#                 for s in spans:
#                     s.merge
        return doc
                

# money_component = MoneyRecognizer(nlp)  # initialise component
# nlp.add_pipe(money_component, first=True)  # add before tagger in the pipeline
nlp.add_pipe("money_amount", before="tagger")

In [15]:
#将 entity识别结果应用到分词
# nlp.add_pipe("merge_noun_chunks")
nlp.add_pipe("merge_entities") #merge一般都在ner之后~

<function spacy.pipeline.functions.merge_entities(doc: spacy.tokens.doc.Doc)>

In [18]:
print(nlp.pipe_names)

['tok2vec', 'money_amount', 'tagger', 'parser', 'attribute_ruler', 'ner', 'global_entityruler', 'merge_entities']


In [19]:
def isInter(a,b):
    result = list(set(a)&set(b))
    if result:
        return True
    else:
        return False

def extract_loader(doc):
    verb_1 = ["承担", "负担", "交纳", "缴纳", "担负"]
    verb_2 = ["赔偿", "退还", "返还", "支付", "赔付", "收取", "迳付", "给付", "退款"]
    loader = []
    
    # 如果是单纯的规定内容，例如”如不服本判决，则当事人需要交上诉费xx元...“，则判定为无效金额
    invalid_cost = True
    for token in doc:
        if token.ent_type_ in ["PARTY", "PARTY_TITLE", "PERSON", "ORG"]:
            invalid_cost = False
            break
            
    if isInter(verb_1, [str(i) for i in doc]):
#         print("processing verb_1...")
        for token in doc:
            if token.dep_ == "nsubj" and token.ent_type_ == "PARTY":
                return [(token.text)]
            if (token.dep_ in ["nmod:prep", "conj", "compound:nn"] and \
                token.ent_type_ in ["PARTY", "PERSON", "ORG", "PARTY_TITLE"]):
#                 print(token.text)
                loader.append(token.text)
#         print(loader)
            
        if len(loader) == 0: #关键字匹配
            party_changed = False
            for token in doc:
                if token.ent_type_ == "PARTY_TITLE":
                    temp_loader = token.text
                    party_changed = True
                if token.ent_type_ == "JUDGE_VERB" and token.text in verb_1:
                    if party_changed:
                        loader.append(temp_loader)
                        party_changed = False
            loader = list(set(loader))
    else:
#         print("processing verb_2...")
        temp_nsubj = []
        #如果句子开头是个介词（例如限），则找ent type 是 party 或 party_title 的 dobj
        if doc[0].dep_ == "dep":
            for token in doc:
                if token.dep_ == "dobj":
                    if token.ent_type_ == "PARTY" or token.ent_type_ == "PARTY_TITLE" \
                    or token.ent_type_ in ["PARTY", "PARTY_TITLE", "ORG", "PROPN", "PERSON", "GPE"]:
                        temp_nsubj.append(token.text)
        else:
            for token in doc:
                if token.dep_ == "nsubj" or token.dep_ == "conj":
                    if token.ent_type_ == "PARTY" or token.ent_type_ == "PARTY_TITLE" \
                    or token.ent_type_ in ["PARTY", "PARTY_TITLE", "ORG", "PROPN", "PERSON", "GPE"]:
                        temp_nsubj.append(token.text)
        if len(temp_nsubj) == 0: 
            for token in doc:
                pass
        elif len(temp_nsubj) == 1:
            loader.append(temp_nsubj[0])
            return loader
        else:
            sents = doc.text.split(",")
            for i in sents:
                if re.search(money_re, i) is not None:
                    temp_doc = nlp(i)
                    for temp_token in temp_doc:
                        if temp_token.dep_ == "nsubj" and temp_token.ent_type_ in ["PARTY", \
                                                                                   "PARTY_TITLE", \
                                                                                   "ORG", "PROPN", \
                                                                                   "PERSON", "GPE"]:
                            loader.append(temp_token.text)
                            break
            if len(loader) == 0:
                loader = [temp_nsubj[0]]
                    
    if invalid_cost: loader = [invalid_cost]   
#     print(loader)
    return loader
            
            

In [20]:
# 处理 judgement result的方法
def extract_judgement_result(t):
#     print("nlp pipelines in extract judgement result method:", nlp.pipe_names)
    t = t.replace("\n", "")
    t = t[:-1] #去掉最后一个符号
    t = re.sub(r'[一二三四五六七八九十]、', '', t)
    t = re.sub(r"\([^()]*\)", "", t)
#     print(t)
    if re.search(money_re, t) is None: return []
    else:
        if "。" in t: sents = t.split("。")
        elif ";" in t: sents = t.split(";")
        else: sents = [t]
        final_result = []
        for i in sents:
            #这里先好好处理单个句子，不考虑是针对某个案件的，先把不同种类的句子都处理好
            #先提取(cost_type or juedge_verb, money_amount)
            temp_extracted_infos = [] #example: [('受理费', '25元', '重庆永辉超市有限公司')]
            judgere_doc = nlp(i)
#             print("judgere_doc: ", judgere_doc)
            if re.search(money_re, judgere_doc.text) is not None:
#                 输出案件结果的实体识别和文本附加信息
#                 displacy.render(judgere_doc, style="ent")
#                 displacy.render(judgere_doc, style="dep")
#                 for token in judgere_doc:
#                     print(token.text, token.pos_, token.dep_, token.ent_type_, token.head.text, token.head.pos_,
#                             [child for child in token.children])
                has_cost_type = False
                e = 0
                while(e < len(judgere_doc.ents)):
                    '''
                    ['支付', '货款', '100元', 'loader']
                    ['减半', '收取', '100元', 'loader']
                    将受理费和减半或者共计同时保留
                    '''
                    d = judgere_doc.ents[e]
                    if d.label_ == "JUDGE_VERB" or d.label_ == "COST_TYPE":
                        has_cost_type = True
                        temp_extracted_info = [d.text, '', '', '']
                        if e != len(judgere_doc.ents)-1:
                            d_1 = judgere_doc.ents[e+1]
                            if d_1.label_ == "COST_TYPE" or d_1.label_ == "JUDGE_VERB":
                                temp_extracted_info[1] = d_1.text
                                e += 1
                        e += 1
                        continue
                        
                    if d.label_ == "MONEY_AMOUNT":
                        if has_cost_type:
                            temp_extracted_info[2] = d.text
                            has_cost_type = False
                        else:
                            temp_extracted_info = ["unknown_type", "unknown_type", d.text, '']
                        temp_extracted_infos.append(temp_extracted_info)
                    e += 1
                
                loader = extract_loader(judgere_doc)
                if len(loader) > 0 and loader[0] == True: 
                    for tr in temp_extracted_infos:
                        tr[3] = []
                else:
                    for tr in temp_extracted_infos:
                        tr[3] = loader
                final_result += temp_extracted_infos
    return final_result
            

In [21]:
# 模块1：当事人信息，与entityruler file的生成有关
def process_one_itext(itext):
    party_info_lst = []
    doc_test = nlp(itext)
#     print(doc_test)
    
    # 先使用token matcher将句子分成以party title开头的文本
    matcher_test = Matcher(nlp.vocab)
    matcher_test.add("PARTY_TITLES", [partytitle_token_pattern])
    matches_test = matcher_test(doc_test)
    temp_party_result = [(doc_test[start:end], (start, end)) for match_id, start, end in matches_test]
#     print(temp_party_result)
    
    for j in range(1, len(matches_test)+1):
        party_info_dict = {}
        if (j == len(matches_test)): 
            temp_end = len(doc_test)
        else: temp_end = matches_test[j][1]
        title = doc_test[matches_test[j-1][1]:matches_test[j-1][2]]
        party_info_dict['title'] = title.text
        
        # processing the info text
        # 如果info里面除了名字还包含其他信息，再处理
        info = doc_test[matches_test[j-1][2]:temp_end]
        
        matcher_info = Matcher(nlp.vocab)
        matcher_info.add("PARTY_INFOS", [partyinfo_token_pattern])
        matches_info = matcher_info(info)
        temp_info_result = [(info[start:end], (start, end)) for match_id, start, end in matches_info]
        
        if len(matches_info) > 0:
            for k in range(1, len(matches_info)+1):
                if (k==len(matches_info)): temp_end_k = len(info)
                else: temp_end_k = matches_info[k][1]
                info_title = info[matches_info[k-1][1]:matches_info[k-1][2]]
                info_content = info[matches_info[k-1][2]:temp_end_k]
                info_content_lst = [info_txt for info_txt in re.split(",|。|:", info_content.text) if len(info_txt) > 0]
                if len(info_content_lst) > 0:
                    content = info_content_lst[0]
                    party_info_dict[str(info_title)] = content
            info = info[:matches_info[0][1]] #将原本的文本按照info关键字切割，前面的部分即为party title相关，后面的为party info相关

        info_basic_lst = [info_txt for info_txt in re.split(",|。|:", info.text) if len(info_txt) > 0]
        if len(info_basic_lst) > 0: party_info_dict['name'] = info_basic_lst[0]
        if len(info_basic_lst) > 1:
            
            for ibtxt in info_basic_lst[1:]:
                #出生日期
                re_date = r"\d{2,4}年\d{1,2}月\d{1,2}日"
                date_m = re.findall(re_date, str(ibtxt))
                if len(date_m) > 0:
                    party_info_dict['birth_date'] = date_m[0]

                #性别
                re_gender = r"男|女"
                gender_m = re.findall(re_gender, str(ibtxt))
                if len(gender_m)>0:
                    party_info_dict['gender'] = gender_m[0]
        
        matcher_occup = Matcher(nlp.vocab)
        matcher_occup.add("PARTY_OCCUPS", [partyoccup_token_pattern])
        matches_occup = matcher_occup(info)
        occup = ''
        if len(matches_occup) > 0:
            occup = info[matches_occup[0][1]: matches_occup[0][2]]
            party_info_dict['occupation'] = str(occup)
        party_info_lst.append(party_info_dict)
#         print(party_info_lst)
#         print()
        
    return party_info_lst
        

In [22]:
#获取当事人信息,将当事人的称号保存至字典中
'''
提取一个案件所有信息的方法,
将不同部分的信息提取按可控模块划分
模块1：当事人信息，与entityruler file的生成有关，如果有时更新了这部分的方法，并且想更新对应的entityruler，则不用继续运行第二个模块
模块2：案件结果信息，与entity ruler file的使用有关，如果有时更新了这部分的方法，并且想修改对应的dict result，则不用更新entityruler file
模块3：其他信息（包含案件原因，法院名称，审判日期等等）
模块4：案件违法行为信息（包含保质期、标签等等关键字匹配）Finished!
后两个模块不论什么时候都需要好好运行
'''

def process_one_case(dict_data, case_id, entityruler, is_entityruler_update, is_dictresult_update, is_case_truth_reason):
    # re-result set part
    party_dicts = {} #案例id：案例当事人信息
    party_dict_list = []
    judge_list = []
    total_judgement_result_list = []
    result_keyword = "unknown"#二审和再审的关键字（维持or撤销），用来判断赢率
    judgeresult_str = [] #案件处理结果的文本list（用作nltk进一步处理）
    case_truth_str = ""
    case_truth_dict = {"不合格产品" : 0, 
                       "违反条款" : 0, 
                       "违反食品安全标准" : 0, 
                       "标签" : 0, 
                       "保质期、生产日期" : 0, 
                       "原材料、添加剂" : 0, 
                       "欺诈" : 0, 
                       "商标违法" : 0, 
                       "出入境检验" : 0, 
                       "生产许可证" : 0,}

    # processing part
    #法院名称
    court_name = dict_data['returnData']['court']
    # print(court_name)

    #审判日期
    judgement_date = dict_data['returnData']['judgementDate']
    # print(judgement_date)

    #审级
    procedure = dict_data['returnData']['procedure']
    # print(procedure)

    #案由
    reason = dict_data['returnData']['reason']
    # print(reason)
    

    for i in dict_data['returnData']['segments']:
        
        #审判人员
        if i['type'] == "JUDGE" and "审" in i['text']:
            judge_list.append(i['text']) #TODO
        
        #处理当事人信息
        if i['typeText'] == '当事人':
#             print("itext: ", i['text'])
            party_dict_lst = process_one_itext(i['text'])
            party_dict_list += party_dict_lst
            for party_dict in party_dict_lst:
                if 'name' in party_dict.keys():
                    entityruler.add_patterns([{"label": "PARTY", "pattern": party_dict['name']}])
                    nlp.tokenizer.pkuseg_update_user_dict([party_dict['name']])
#       审判结果
        if is_dictresult_update:
            if i['type'] == "JUDGEMENT_RESULT":
                if "维持" in i['text']: result_keyword = "维持"
                if "撤销" in i['text']: result_keyword = "撤销"
                judgeresult_str.append(i['text'])
                temp_results = extract_judgement_result(i['text'])
                if len(temp_results) > 0:
                    total_judgement_result_list.append(temp_results)
                    
        if i['type'] == "ASCERTAIN" or i['type'] == "EVIDENCE" or i['type'] == "COUNTER_WORDS"\
         or i['type'] == "COURT_HELD" or i['type'] == "courtHeld":
            case_truth_str += i['text']
    
    if is_case_truth_reason:
        ct_doc = nlp(case_truth_str)
        ct_temp = ct_matcher(ct_doc)
#         print(case_id)
#         print(case_truth_str)
        if len(ct_temp) > 0:
            for match_id, start, end in ct_temp:
                string_id = nlp.vocab.strings[match_id]  # Get string representation
#                 print(string_id, type(string_id))
                case_truth_dict[string_id] += 1
#         print()
        return case_id, case_truth_dict
    
    
    # 提取审判人员信息（取第一个人的名字即可）
    party_dicts['judge'] = ""
    if len(judge_list) > 0:
        judge_text = judge_list[0].replace(" ", "")
        matcher_jt = PhraseMatcher(nlp.vocab)
        patterns_jtitles = [nlp.make_doc(text) for text in judge_titles]
        matcher_jt.add("JUDGE_TITLE", patterns_jtitles)
        doc_jt = nlp(judge_text)
        matches_jt = matcher_jt(doc_jt)
        if len(matches_jt) > 0:
            match_list = [(start, end) for match_id, start, end in matches_jt]
            start = match_list[0][1]
            end = len(doc_jt)
            for i in range(1, len(match_list)):
                temp_end = match_list[i][0]
                if temp_end > start: 
                    end = temp_end
                    break
            final_judge = doc_jt[start:end].text
            party_dicts['judge'] = final_judge
        else: party_dicts['judge'] = judge_text
        
        
#     print(party_dict_list)
    party_dicts["case_id"] = case_id
    party_dicts["party_info"] = party_dict_list
    party_dicts["judgement_date"] = judgement_date
    party_dicts["procedure"] = procedure
    party_dicts["reason"] = reason
    party_dicts["court_name"] = court_name
    party_dicts['judgement_result'] = total_judgement_result_list
    party_dicts['judgere_keyword'] = result_keyword
    
    if is_entityruler_update: return entityruler
    
    return party_dicts, judgeresult_str




In [23]:
# tt_lst = [
#     '原告:薛焱锋,男,1984年11月15日出生,汉族,无固定职业,住乌鲁木齐市西北路被告:陕西西凤酒股份有限公司,住所地:陕西省凤翔县柳林镇。', 
#     '法定代表人:秦本平,该公司董事长。', 
#     '委托代理人:孙海平,男,1966年7月29日出生,汉族,该公司法务部部长,住陕西省委托代理人:杨阳,男,1987年9月19日出生,汉族,该公司法务部职员,住陕西省被告:新疆友好百盛商业发展有限公司,住所地:新疆乌鲁木齐市友好南路668号。', 
#     '法定代表人:李成,该公司总经理。', 
#     '原告:张建纯,女,1963年1月28日出生,汉族,住四川省蓬安县,委托代理人:裴紫薇,广东普罗米修(广州)律师事务所律师。', 
#     '被告:华润万家生活超市(中山)有限公司,住所地广东省中山市石岐区中山五路,组织机构代码72651103-7。', 
#     '原告冯志波,男,1970年5月17日出生,汉族。', 
#     '被告武汉生达新世界大药房有限公司民意店,住所地:武汉市硚口区中山大道285号民意商城1、2号楼1层1室。', 
#     '原告徐利波,男,汉族,1982年8月21日出生,住四川省浦江县西来镇铜鼓村14组。', 
#     '被告四川华润万家好来超市有限公司高新区店。住所地:成都高新区府城大道中段88号。', 
#     '原告:王福岭,男,1984年10月25日出生,汉族,住山东省曹县,被告:东莞市虎门喜之宝妇婴用品店。住所地:广东省东莞市虎门镇港口路能源华庄二区首层B110号铺,组织机构代码为L3439207-6。', 
#     '原告吴剑兵,男,汉族,1974年9月10日出生,住址广东省深圳市福田区,委托代理人陈书伟,男,汉族,1972年10月15日出生,身份证住址广东省深圳市龙岗区,系原告所在公司深圳市操歌文化传播有限公司员工。', 
#     '被告北京华联精品超市有限公司东直门店,住所地北京市东城区东直门南大街1号北京来福士中心地下一层。', 
#     '被告北京海汇百家超市,住所地北京市昌平区东小口镇新干线家园一区1号楼1层02。', 
#     '原告李冰', 
#     '系原告李冰', 
#     '法定代表人:林旭儒以上两被告的共同委托代理人:邓梅,女,1979年9月1日出生,壮族,该公司员工,住南宁市。',
#     '委托代理人陈江河,系原告冬梅的丈夫。', 
#     '法定代表人龙涛,该公司负责人。', 
#     '被告佛山市新一佳百货超市有限公司禅城荟丽店,住所地,注册号(分)440602000112609。', 
#     '原告骆永明,男,汉族,出生于1974年10月8日,西安市长安区王莽乡。'
# ]

# for i in tt_lst[-1:]:
#     print(process_one_itext(i))

In [24]:
# 对多个json文件试一下
def process_batch_cases(dir_path, entityruler, is_entityruler_update, is_dictresult_update, is_case_truth_reason):
    if not os.path.exists(dir_path):
        print("路径不存在！", dir_path)
        return
    case_count = 0
    case_dict_lists = [] #一个文件夹下的案例处理后要存入的集合
    case_truth_dlist = {}
    judgeresult_strs = []
#     keyword_text_lists = []
    print("----------------------------executing path: ", dir_path)

    # path = "/Users/starice/OwnFiles/cityu/RA/type1/2014/3/json" #文件夹目录
    files = os.listdir(dir_path) #得到文件夹下的所有文件名称
    for file in files: #遍历文件夹
        if os.path.splitext(file)[-1][1:] != "json": continue
        case_count += 1
        case_id = file.split(".")[0]
        if not os.path.isdir(file): #判断是否是文件夹，不是文件夹才打开
            with open(dir_path + "/" + file, 'r', encoding="UTF-8") as f: data = json.load(f)
            dict_data = json.loads(data)
            if is_entityruler_update:
                entityruler = process_one_case(dict_data, 
                                                case_id, 
                                                entityruler, 
                                                is_entityruler_update, 
                                                is_dictresult_update, 
                                               is_case_truth_reason)
#                 if _entityruler is not None: return _entityruler
            if is_dictresult_update:
                _party_dicts, _judgeresult_str = process_one_case(dict_data, case_id, 
                                                                  entityruler, 
                                                                  is_entityruler_update, 
                                                                  is_dictresult_update, 
                                                                   is_case_truth_reason)
                case_dict_lists.append(_party_dicts)
                judgeresult_strs += _judgeresult_str
            if is_case_truth_reason:
                case_id, _ct_list = process_one_case(dict_data, 
                                                case_id, 
                                                entityruler, 
                                                is_entityruler_update, 
                                                is_dictresult_update,  
                                                is_case_truth_reason)
                case_truth_dlist[case_id] = _ct_list
                
    if is_case_truth_reason: return case_truth_dlist
    
    if (is_entityruler_update and entityruler is not None): return entityruler
    
    return case_dict_lists, judgeresult_strs

In [25]:
base_url = "/Users/starice/OwnFiles/cityu/RA/"
pre_dir = ['type1', 'type2', 'type3', 'type4']
dir_name = ['2014', '2015', '2016', '2017', '2018', '2019', '2020']
dir_sname = ['1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12']
party_titles = set()
party_info_titles = set()
total_judgement_result = []

def switch_dict_0(pre_dir, dir_name, dir_sname, entityruler):
    entityruler.to_disk(total_basic_url + "party_entityruler_files/" + "entityruler_patterns_" + str(pre_dir) + "_" + str(dir_name) + "_" + str(dir_sname) + ".jsonl")

def switch_dict_1(pre_dir, dir_name, dir_sname, _case_dict_lists):
    with open(total_basic_url + "total_dict_result/" + "dict_result_" + str(pre_dir) + "_" + str(dir_name) + "_" + str(dir_sname) + ".txt", 'w', encoding="UTF-8") as f:
        for case_dict in _case_dict_lists:
            f.write(str(case_dict) + "\n")
    name = "party_entityruler_"+str(pre_dir)+"_"+str(dir_name)+"_"+str(dir_sname)
    nlp.remove_pipe(name)
    
def switch_default():
    print("This method is just for switch dict structure.")

def process_multiple_cases(pre_dir, dir_name, dir_sname, is_entityruler_update, is_dictresult_update, is_case_truth_reason):
    
    total_count = 0
    
    for i in pre_dir:
        for j in dir_name:
            for k in dir_sname:
                
                url = base_url + i + "/" + j + "/" + k + "/json"
                config={"overwrite_ents": True}
                entityruler = EntityRuler(nlp, overwrite_ents=True)
                
                if is_entityruler_update:
                    _entityruler = process_batch_cases(url, 
                                                       entityruler, 
                                                       is_entityruler_update, 
                                                       is_dictresult_update, 
                                                        is_case_truth_reason)
                    if _entityruler is not None:
                        switch_dict_0(i, j, k, _entityruler) 
                
                if is_dictresult_update:  
#                   处理哪部分case文件就引入哪部分的party patterns
                    party_entityruler_pattern_path = total_basic_url + "party_entityruler_files/" + "entityruler_patterns_" + str(i) + "_" + str(j) + "_" + str(k) + ".jsonl"
                    if not os.path.exists(party_entityruler_pattern_path):
                        print("Entityruler patterns file 路径不存在！使用默认的entity ruler可能会影响金额承担者的提取！", party_entityruler_pattern_path)
                    else:
                        name = "party_entityruler_"+str(i)+"_"+str(j)+"_"+str(k)
                        if name not in list(nlp.pipe_names):
                            party_entityruler = nlp.add_pipe("entity_ruler", name=name, after="ner", config=config)
                            party_entityruler.from_disk(party_entityruler_pattern_path)
                
                    process_batchcases_result = process_batch_cases(url, 
                                                                    entityruler, 
                                                                   is_entityruler_update, 
                                                                   is_dictresult_update, 
                                                                   is_case_truth_reason)
                    if process_batchcases_result is not None:
                        _case_dict_lists, _judgeresult_strs = process_batchcases_result[0], process_batchcases_result[1]
                        switch_dict_1(i, j, k, _case_dict_lists)
#                         return _judgeresult_strs

                    else: print("Unsuccessfully processed directory: ", url, " !")
        
                if is_case_truth_reason:
                    _case_truth_dlist = process_batch_cases(url, 
                                                            entityruler, 
                                                            is_entityruler_update, 
                                                            is_dictresult_update, 
                                                           is_case_truth_reason)
                    with open(total_basic_url + "total_dict_result/" + "case_truth_reason_result_" + str(i) + "_" + str(j) + "_" + str(k) + ".txt", 'w', encoding="UTF-8") as f:
                        f.write(str(_case_truth_dlist))

In [ ]:
process_multiple_cases(pre_dir[1:2], dir_name[3:4], dir_sname[3:4], False, False, True)
# print(nlp.pipe_names)

----------------------------executing path:  /Users/starice/OwnFiles/cityu/RA/type2/2017/4/json


<h2>测试用例部分：用来单独测试某一模块方法</h2>
这里以type4 2019 1为样例数据

In [39]:
# name = "party_entityruler_"+str("type4")+"_"+str("2019")+"_"+str(1)
# party_entityruler_pattern_path = total_basic_url + "party_entityruler_files/" + "entityruler_patterns_" + str("type4") + "_" + str("2019") + "_" + str("1") + ".jsonl"
# config={"overwrite_ents": True}
# nlp.remove_pipe(name)
# if name not in list(nlp.pipe_names):
#     party_entityruler = nlp.add_pipe("entity_ruler", name=name, config=config)
#     party_entityruler.from_disk(party_entityruler_pattern_path)
# print(nlp.pipeline[-1][1].patterns)

In [177]:
# 目前存在的问题
# 1. 惩罚金的提取：有时候是驳回原告的惩罚金请求，这时出现的金额不应该被提取出来✅
# 2. 有时是受理费共计多少，应该提取受理费而不是共计，可以考虑将所有的金额类型关键字都保留，这样应该会好一些✅
# 3. 有时受理费是先由原告预交，这个也要判断一下✅
# 4. 为了更好的匹配，需要将最终处理的字段做去标点符号以及不必要括号内容的处理✅
# 5. cost verb新增： 担负✅
# 6. 十倍价款，十倍货款等类似表达都是惩罚金✅
# 7. 统一赔偿费用应该改为unknown type，不一定所有的统一赔偿费用都是赔偿款项✅

<h3>测试用例1：案件结果金额以及承担人提取</h3>
<p>一审案件结果已经重新定义如下：<br>如果原告没有拿到钱则为输<br>如果原告只拿到货款或者又有货款又有惩罚金则为赢</p>
<h4><font color="red">所以与之前重点提取案件受理费由谁承担不同，这次要重点确保金额类型的准确程度，金额承担人的准确程度在其次</font></h4>

In [155]:
# judgere_txt = [
#     "一、被告常州鼎圣堂生物科技有限公司向原告翁艳蕾退还货款3600元;",
#     "二、原告翁艳蕾向被告常州鼎圣堂生物科技有限公司退还“鼎圣堂牌”虫草花80罐(如不能返还实物,则应从退还的货款中扣除相应货款);",
#     "三、被告常州鼎圣堂生物科技有限公司向原告翁艳蕾支付赔偿金36000元。",
#     "如果未按本判决指定的期间履行给付金钱义务,应当依照《中华人民共和国民事诉讼法》第二百五十三条规定,加倍支付迟延履行期间的债务利息。",
#     "案件受理费790元,减半收取计395元,由被告常州鼎圣堂生物科技有限公司负担(此款原告翁艳蕾已垫付法院,被告常州鼎圣堂生物科技有限公司随判决付款一并给付原告翁艳蕾)。\
#     如不服本判决,可在判决书送达之日起十五日内向本院递交上诉状,并按对方当事人或者代表人的人数提出副本上诉于湖北省武汉市中级人民法院。", 
#     "一、撤销深圳市南山区人民法院(2017)粤0305民初18194号民事判决;", 
#     "二、被上诉人黄东鸿应于本判决生效之日起十日内支付上诉人徐林斌赔偿金52680元。", 
#     "如果未按本判决指定的期间履行给付金钱义务,应当依照《中华人民共和国民事诉讼法》第二百五十三条之规定,加倍支付迟延履行期间的债务利息。", 
#     "一审案件受理费558.5元,公告费900元,均由被上诉人黄东鸿负担。", 
#     "二审案件受理费1248.70元,由被上诉人黄东鸿负担。本判决为终审判决。", 
#     "驳回原告刘博逊的全部诉讼请求。", 
#     "案件受理费25元,由原告刘博逊负担(已交纳)。", 
#     "如不服本判决,可以在判决书送达之日起十五日内,向本院递交上诉状,并按对方当事人或者代表人的人数提出副本,上诉于北京市第二中级人民法院。", 
#     "一、被告杨洲于本判决生效之日起十日内退还原告赵磊购物款1400元;", 
#     "二、原告赵磊于本判决生效之日起十日内将所购买的完整包装产品“日本#IB院线ag抗糖酵素原液”4盒共8瓶退还被告杨洲(如不能返还实物,则应从退还的货款中扣除相应货款);",
#     "三、被告杨洲于本判决生效之日起十日内向原告赵磊支付赔偿款14000元;",
#     "如果未按本判决指定的期间履行给付金钱义务,应当依照《中华人民共和国民事诉讼法》第二百五十三条规定,加倍支付迟延履行期间的债务利息。",
#     "案件受理费93元,由被告杨洲负担(于本判决生效后七日内交纳)。",
#     "如不服本判决,可以在判决书送达之日起十五日内,向本院递交上诉状,并按对方当事人的人数提出副本,上诉于北京市第四中级人民法院。",
#     "一、解除刘锐亮与北京沃尔玛百货有限公司知春路分店的买卖合同,刘锐亮于本判决生效后七日内退还北京沃尔玛百货有限公司知春路分店“金锣火腿肠”一袋,北京沃尔玛百货有限公司知春路分店于本判决生效后七日内退还刘锐亮货款六元五角;",
#     "二、北京沃尔玛百货有限公司知春路分店于本判决生效后七日内给付刘锐亮赔偿金一千元。",
#     "如果未按本判决指定的期间履行给付金钱义务,应当依照《中华人民共和国民事诉讼法》第二百五十三条之规定,加倍支付迟延履行期间的债务利息。",
#     "案件受理费二十五元(刘锐亮已预交),由北京沃尔玛百货有限公司知春路分店负担,于本判决书生效后七日内交纳。",
#     "如不服本判决,可在判决书送达之日起十五日内向本院递交上诉状,并按对方当事人的人数提出副本,上诉于北京市第一中级人民法院。",
#     "一、原告陈明江于本判决生效之日起十日内退还被告深圳市吉百家商贸有限公司MALAVIDA22瓶,商品条形码为8413513690699和8413513690682,被告深圳市吉百家商贸有限公司在收到退货同时向原告陈明江支付货款1616元;",
#     "二、驳回原告的其他诉讼请求。",
#     "若被告未按本判决指定的期间履行给付金钱的义务,应当按照《中华人民共和国民事诉讼法》第二百五十三条的规定,加倍支付迟延履行期间的债务利息。",
#     "本案诉讼费122元,由被告承担11元,原告承担111元。上述费用原告已预交。",
#     "如不服本判决,可在判决书送达之日起十五日内,向本院递交上诉状,并按对方当事人的人数提出副本,上诉于广东省深圳市中级人民法院。",
#     "案件受理费244元,由喻忠负担187元,由广东粤北行健康食品有限公司负担57元。广东粤北行健康食品有限公司负担部分案件受理费于本判决生效之日起七日内交纳本院。", 
#     "一、广东粤北行健康食品有限公司退还喻忠货款374.48元,赔偿十倍价款3744.8元,合计4119.28元,该款于本判决生效之日起十日内付清。", 
#     "一、被告广东粤北行健康食品有限公司退还喻忠货款374.48元,赔偿价款的十倍3744.8元,合计4119.28元,该款于本判决生效之日起十日内付清。", 
#     "一、被告广东粤北行健康食品有限公司退还喻忠货款374.48元,赔偿3744.8元,合计4119.28元,该款于本判决生效之日起十日内付清。", 
# ]

In [36]:
# for i in judgere_txt:
# #     print(i)
#     print(extract_judgement_result(i))
#     print()

<h3>测试用例2：案件违法原因提取</h3>

In [169]:
test_text = '本院认为本案当事人的争议焦点主要为:步步高公司、步步高星沙店是否应当承担食品价款十倍赔偿责任。\
经审理查明,本案涉案的品香府劲道豆干预包装标示产品包装标示的营养成分表载明每100g中能量156KJ、\
蛋白质14.9g、脂肪9.1g、碳水化合物4.0g、钠90.3mg,依照中国人民共和国国家卫生和计划生育委员\
会发布的《预包装食品营养标签通则》(GB28050-2011)问答(修订版)第二十三条关于能量及其折算的\
解释规定,根据涉案产品营养成分表所标示的蛋白质、脂肪、碳水化合物含量折算,能量应为658KJ。\
根据《中华人民共和国食品安全法》第七十一条第三款:“食品和食品添加剂与其标签、说明书的内容不符的\
,不得上市销售”以及《食品安全国家标准预包装食品营养标签通则》第3.1条“预包装食品营养标签标示的\
任何营养信息,应真实、客观,不得标示虚假信息,不得夸大产品的营养作用或其他作用”之规定,该产品未标注好营养成分,且未通过出入境检验,\
属于违反食品安全标准的产品。'

ct_doc = nlp(test_text)
ct_temp = ct_matcher(ct_doc)
print(ct_temp)
if len(ct_temp) > 0:
    print(1)
    for match_id, start, end in ct_temp:
        string_id = nlp.vocab.strings[match_id]  # Get string representation
        print(string_id, type(string_id), ct_doc[start:end], "\n")

ct_matcher_p3 = Matcher(nlp.vocab)
ct_matcher_p3.add("违反食品安全标准", [ct_pattern_3])
ct_temp = ct_matcher_p3(ct_doc)
if len(ct_temp) > 0:
    print(1)
    for match_id, start, end in ct_temp:
        string_id = nlp.vocab.strings[match_id]  # Get string representation
        print(string_id, type(string_id), ct_doc[start:end], "\n")


[(5226792321437755001, 74, 75), (4412146379254569920, 131, 132), (5226792321437755001, 133, 134), (5226792321437755001, 158, 159), (5226792321437755001, 190, 194), (14360318142141649319, 198, 199), (15293864229149315260, 202, 204)]
1
标签 <class 'str'> 标签 

原材料、添加剂 <class 'str'> 添加剂 

标签 <class 'str'> 标签 

标签 <class 'str'> 标签 

标签 <class 'str'> 未标注好营养成分 

出入境检验 <class 'str'> 出入境检验 

违反食品安全标准 <class 'str'> 违反食品安全 

1
违反食品安全标准 <class 'str'> 违反食品安全 

